In [6]:
clear all

In [1]:
ssc install estout, replace

In [7]:
use "~/Documents/Working_Directory/Tex_MD/Teaching/tutorial/6002/W7/airfare.dta", clear

# set time and individual ID

In [8]:
xtset id year

       panel variable:  id (strongly balanced)
        time variable:  year, 1997 to 2000
                delta:  1 unit


# compute the descriptive statistic in this panel

In [9]:
xtsum id year lfare concen ldist


Variable         |      Mean   Std. Dev.       Min        Max |    Observations
-----------------+--------------------------------------------+----------------
id       overall |       575   331.7237          1       1149 |     N =    4596
         between |              331.832          1       1149 |     n =    1149
         within  |                    0        575        575 |     T =       4
                 |                                            |
year     overall |    1998.5   1.118156       1997       2000 |     N =    4596
         between |                    0     1998.5     1998.5 |     n =    1149
         within  |             1.118156       1997       2000 |     T =       4
                 |                                            |
lfare    overall |  5.095601   .4363999   3.610918   6.257668 |     N =    4596
         between |              .425127   4.026386   6.127917 |     n =    1149
         within  |              .099146   4.176027   6.077457 |     T =

# (a) : Pooled OLS

In [27]:
reg lfare y98 y99 y00 concen ldist ldistsq

estimates store OLS



      Source |       SS           df       MS      Number of obs   =     4,596
-------------+----------------------------------   F(6, 4589)      =    523.18
       Model |  355.453858         6  59.2423096   Prob > F        =    0.0000
    Residual |  519.640516     4,589  .113236112   R-squared       =    0.4062
-------------+----------------------------------   Adj R-squared   =    0.4054
       Total |  875.094374     4,595  .190444913   Root MSE        =    .33651

------------------------------------------------------------------------------
       lfare |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         y98 |   .0211244   .0140419     1.50   0.133    -.0064046    .0486533
         y99 |   .0378496   .0140413     2.70   0.007      .010322    .0653772
         y00 |     .09987   .0140432     7.11   0.000     .0723385    .1274015
      concen |   .3601203   .0300691    11.98   0

# check if there is heteroskedasticity (using DW test and BP test) with $$H_0: \mbox{ there is no heteroskedasticity}$$

In [20]:
estat hettest


Breusch-Pagan / Cook-Weisberg test for heteroskedasticity 
         Ho: Constant variance
         Variables: fitted values of lfare

         chi2(1)      =   389.62
         Prob > chi2  =   0.0000


In [21]:
estat imtest, white


White's test for Ho: homoskedasticity
         against Ha: unrestricted heteroskedasticity

         chi2(20)     =    983.66
         Prob > chi2  =    0.0000

Cameron & Trivedi's decomposition of IM-test

---------------------------------------------------
              Source |       chi2     df      p
---------------------+-----------------------------
  Heteroskedasticity |     983.66     20    0.0000
            Skewness |     165.39      6    0.0000
            Kurtosis |     117.63      1    0.0000
---------------------+-----------------------------
               Total |    1266.67     27    0.0000
---------------------------------------------------


# both H0 are rejected; hence, we need robust se

In [26]:
reg lfare y98 y99 y00 concen ldist ldistsq, robust

estimates store OLS_rb



Linear regression                               Number of obs     =      4,596
                                                F(6, 4589)        =     558.39
                                                Prob > F          =     0.0000
                                                R-squared         =     0.4062
                                                Root MSE          =     .33651

------------------------------------------------------------------------------
             |               Robust
       lfare |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         y98 |   .0211244   .0141734     1.49   0.136    -.0066623     .048911
         y99 |   .0378496   .0144012     2.63   0.009     .0096162    .0660829
         y00 |     .09987   .0143821     6.94   0.000     .0716742    .1280658
      concen |   .3601203   .0318147    11.32   0.000     .2977482    .4224925
       ldist 

# let's compare two methods

In [28]:
estout OLS OLS_rb, cells(b(star fmt(3)) se(par fmt(3))) stats(r2 F chi2 N, labels(R-squared F-value Chi2-value "sample size")) legend


--------------------------------------------
                      OLS          OLS_rb   
                     b/se            b/se   
--------------------------------------------
y98                 0.021           0.021   
                  (0.014)         (0.014)   
y99                 0.038**         0.038** 
                  (0.014)         (0.014)   
y00                 0.100***        0.100***
                  (0.014)         (0.014)   
concen              0.360***        0.360***
                  (0.030)         (0.032)   
ldist              -0.902***       -0.902***
                  (0.128)         (0.141)   
ldistsq             0.103***        0.103***
                  (0.010)         (0.010)   
_cons               6.209***        6.209***
                  (0.421)         (0.471)   
--------------------------------------------
R-squared           0.406           0.406   
F-value           523.175         558.395   
Chi2-value                                  
sample si

# (d): RE

In [30]:
xtreg lfare y98 y99 y00 concen ldist ldistsq, re

estimates store RE_LS



Random-effects GLS regression                   Number of obs     =      4,596
Group variable: id                              Number of groups  =      1,149

R-sq:                                           Obs per group:
     within  = 0.1348                                         min =          4
     between = 0.4176                                         avg =        4.0
     overall = 0.4030                                         max =          4

                                                Wald chi2(6)      =    1360.42
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.0000

------------------------------------------------------------------------------
       lfare |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         y98 |   .0224743   .0044544     5.05   0.000     .0137438    .0312047
         y99 |   .0366898   .0044528     8.24   0.000     .0279

# RE with robust se

In [24]:
xtreg lfare y98 y99 y00 concen ldist ldistsq, re robust

estimates store RE_rb



Random-effects GLS regression                   Number of obs     =      4,596
Group variable: id                              Number of groups  =      1,149

R-sq:                                           Obs per group:
     within  = 0.1348                                         min =          4
     between = 0.4176                                         avg =        4.0
     overall = 0.4030                                         max =          4

                                                Wald chi2(6)      =    1222.73
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.0000

                                 (Std. Err. adjusted for 1,149 clusters in id)
------------------------------------------------------------------------------
             |               Robust
       lfare |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         y98 |   .0224743  

# (e): FE

In [31]:
xtreg lfare y98 y99 y00 concen ldist ldistsq, fe

estimates store FE_LS


note: ldist omitted because of collinearity
note: ldistsq omitted because of collinearity

Fixed-effects (within) regression               Number of obs     =      4,596
Group variable: id                              Number of groups  =      1,149

R-sq:                                           Obs per group:
     within  = 0.1352                                         min =          4
     between = 0.0576                                         avg =        4.0
     overall = 0.0083                                         max =          4

                                                F(4,3443)         =     134.61
corr(u_i, Xb)  = -0.2033                        Prob > F          =     0.0000

------------------------------------------------------------------------------
       lfare |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         y98 |   .0228328   .0044515     5.13   0.00

# FE with robust se

In [25]:
xtreg lfare y98 y99 y00 concen ldist ldistsq, fe robust

estimates store FE_rb


note: ldist omitted because of collinearity
note: ldistsq omitted because of collinearity

Fixed-effects (within) regression               Number of obs     =      4,596
Group variable: id                              Number of groups  =      1,149

R-sq:                                           Obs per group:
     within  = 0.1352                                         min =          4
     between = 0.0576                                         avg =        4.0
     overall = 0.0083                                         max =          4

                                                F(4,1148)         =     120.06
corr(u_i, Xb)  = -0.2033                        Prob > F          =     0.0000

                                 (Std. Err. adjusted for 1,149 clusters in id)
------------------------------------------------------------------------------
             |               Robust
       lfare |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+--

# (f) test whether FE model and RE model are the same in population

# we cannot use Hausman test for models with robust se

In [11]:
hausman FE_LS RE_LS


                 ---- Coefficients ----
             |      (b)          (B)            (b-B)     sqrt(diag(V_b-V_B))
             |     FE_LS        RE_LS        Difference          S.E.
-------------+----------------------------------------------------------------
         y98 |    .0228328     .0224743        .0003585               .
         y99 |    .0363819     .0366898        -.000308               .
         y00 |    .0977717      .098212       -.0004403               .
      concen |     .168859     .2089935       -.0401345        .0126937
------------------------------------------------------------------------------
                           b = consistent under Ho and Ha; obtained from xtreg
            B = inconsistent under Ha, efficient under Ho; obtained from xtreg

    Test:  Ho:  difference in coefficients not systematic

                  chi2(4) = (b-B)'[(V_b-V_B)^(-1)](b-B)
                          =       10.00
                Prob>chi2 =      0.0405
           

# now let's compare everything

In [33]:
estout FE_LS FE_rb RE_LS RE_rb, cells(b(star fmt(3)) se(par fmt(3))) stats(r2 F chi2 N, labels(R-squared F-value Chi2-value "sample size")) legend


----------------------------------------------------------------------------
                    FE_LS           FE_rb           RE_LS           RE_rb   
                     b/se            b/se            b/se            b/se   
----------------------------------------------------------------------------
y98                 0.023***        0.023***        0.022***        0.022***
                  (0.004)         (0.004)         (0.004)         (0.004)   
y99                 0.036***        0.036***        0.037***        0.037***
                  (0.004)         (0.005)         (0.004)         (0.005)   
y00                 0.098***        0.098***        0.098***        0.098***
                  (0.004)         (0.006)         (0.004)         (0.006)   
concen              0.169***        0.169***        0.209***        0.209***
                  (0.029)         (0.049)         (0.027)         (0.042)   
ldist               0.000           0.000          -0.852***       -0.852**

In [35]:
!rm -rf W7_Stata.html
!jupyter nbconvert --to html W7_Stata.ipynb





[NbConvertApp] Converting notebook W7_Stata.ipynb to html
[NbConvertApp] Writing 592171 bytes to W7_Stata.html
